In [20]:
import pandas as pd
import numpy as np
import json, datetime

In [27]:
countries_region = pd.read_csv("countries_region.csv")

In [22]:

months = ['jan20', 'feb20', 'mar20', 'apr20', 'may20', 'jun20', 'jul20', 'aug20', 'sep20', 'oct20', 'nov20', 'dec20',
    'jan21', 'feb21', 'mar21', 'apr21', 'may21', 'jun21', 'jul21', 'aug21', 'sep21', 'oct21', 'nov21']
months_df = {}
for month in months:
    tmp = pd.read_csv("standard_{}frl.csv".format(month), sep = ';')
    months_df[month] = tmp[tmp['flag'] != 'wi'][tmp['flag'] != 'i'][['country', 'title', 'rating']]

/tmp/ipykernel_147199/2183709019.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  months_df[month] = tmp[tmp['flag'] != 'wi'][tmp['flag'] != 'i'][['country', 'title', 'rating']]


In [23]:
def replace_title(x):
    if x[1] in ['WFM', 'IM', 'GM', 'WIM', 'FM', 'WGM', 'CM', 'WCM']:
        x[1] = 1
    else:
        x[1] = 0
    return x

In [24]:
intermediate_result = pd.DataFrame(columns=['country', 'month', 'nbr_players', 'top_k_avg', 'nbr_titles'])

for month in months_df:
    tmp = months_df[month]
    tmp = tmp.apply(replace_title, axis=1)
    
    # Calculate top 10 avg rating per federation
    top_k_avg = tmp.groupby(['country'], sort=True) \
        .apply(lambda x: x.sort_values(['rating'], ascending=False)) \
        .reset_index(drop=True)
    top_k_avg = top_k_avg.groupby('country').head(10)
    top_k_avg = top_k_avg.groupby(['country'], as_index=False).agg({'rating': 'mean'})

    # Calculate number of players and number of titles per federation
    tmp['eff'] = 1
    nbr_players_titles = tmp.groupby(['country'], as_index=False).agg({'title': 'sum', 'eff': 'sum'})

    # Merge the whole
    tmp = top_k_avg.merge(nbr_players_titles, how='inner', on='country')
    tmp.columns = ['country', 'top_k_avg', 'nbr_titles', 'nbr_players']
    tmp['month'] = month
    intermediate_result = intermediate_result.append(tmp)

In [26]:
set(intermediate_result.country.unique()) - set(countries_region.country.unique())

{'CAM'}

In [28]:
final_data = []
for country in intermediate_result.country.unique():
    bla = intermediate_result[intermediate_result.country == country]
    country_dict = {
        "name": country
        , "region": countries_region[countries_region.country==country].region.unique()[0]
    }
    country_dict['nbr_players'] = bla.nbr_players.to_list()
    country_dict['top_k_avg'] = bla.top_k_avg.to_list()
    country_dict['nbr_titles'] = bla.nbr_titles.to_list()
    final_data.append(country_dict)


[{'name': 'AFG',
  'region': 'Asie',
  'nbr_players': [28,
   22,
   23,
   22,
   22,
   22,
   22,
   21,
   17,
   14,
   30,
   30,
   29,
   23,
   23,
   22,
   22,
   23,
   23,
   22,
   18,
   15,
   15],
  'top_k_avg': [1903.0,
   1841.0,
   1846.9,
   1847.2,
   1847.0,
   1847.0,
   1847.0,
   1841.7,
   1793.6,
   1760.2,
   1907.1,
   1907.0,
   1907.3,
   1844.4,
   1843.3,
   1839.8,
   1839.5,
   1867.1,
   1868.2,
   1866.7,
   1824.9,
   1801.8,
   1791.5],
  'nbr_titles': [4,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   1,
   1,
   4,
   4,
   4,
   2,
   2,
   2,
   2,
   3,
   3,
   3,
   2,
   2,
   2]},
 {'name': 'AHO',
  'region': 'Amérique',
  'nbr_players': [28,
   25,
   24,
   23,
   23,
   22,
   22,
   22,
   20,
   20,
   28,
   28,
   28,
   25,
   24,
   23,
   23,
   22,
   22,
   22,
   20,
   20,
   21],
  'top_k_avg': [2031.9,
   1971.6,
   1971.2,
   1946.5,
   1946.5,
   1935.2,
   1935.2,
   1935.2,
   1935.2,
   1935.2,
   2030.7,
   2030.7,
  

In [8]:
def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
with open('feds_standard_evolution.json', 'w') as outfile:
    json.dump(final_data, outfile, default=myconverter)

In [9]:
countries_region.region.unique()

array(['Asie', 'Europe', 'Afrique', 'Océanie', 'Amérique', 'Iconnu'],
      dtype=object)

In [19]:
intermediate_result[intermediate_result.country=='IND']

,country,month,nbr_players,top_k_avg,nbr_titles
73,IND,jan21,12451,2669.6,328
73,IND,feb21,11843,2667.5,324
73,IND,mar21,11738,2667.3,326
72,IND,apr21,11806,2667.0,328
72,IND,may21,11372,2667.0,327
71,IND,jun21,10356,2667.0,327
71,IND,jul21,9405,2667.0,323
70,IND,aug21,9010,2670.5,314
67,IND,sep21,8495,2669.2,300
67,IND,oct21,8225,2671.4,302
